In [56]:
import torch
import os
from torchvision import datasets, transforms, models

In [57]:
train_dir = os.path.join('dataset', 'part_one_dataset', 'train_data')
eval_dir = os.path.join('dataset', 'part_one_dataset', 'eval_data')

In [58]:
train_path = os.path.join(train_dir, '1_train_data.tar.pth')
eval_path = os.path.join(eval_dir, '1_eval_data.tar.pth')

t = torch.load(train_path, weights_only = False)

In [60]:
from torchvision import models
import torch

# Load a pre-trained ResNet model
resnet =  models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])  # Remove the last layer
resnet.eval()  # Set to evaluation mode

# Move model to GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cpu'
resnet = resnet.to(device)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 (ResNet input size)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])


In [61]:
domains = [{} for _ in range(10)]

for j in range(10):
    
    train_path = os.path.join(train_dir, f'{j+1}_train_data.tar.pth')
    t = torch.load(train_path, weights_only = False)

    data = t['data'] # both numpy.ndarray
    
    domains[j]['labels'] = t['targets'] if 'targets' in t else None
    
    try:
        domains[j]['features']  = torch.load(f'stuff/train_embeds_{j+1}.pt', map_location = device)
        # embeds = torch.stack(embeds).to(device)
    except: 
        embeds = []
        # Convert to PyTorch tensor
        X_tensor = torch.tensor(data, dtype=torch.float32)  # Convert to tensor
        X_tensor = X_tensor.permute(0, 3, 1, 2)  # Change shape to (2500, 3, 32, 32)

        tensor = X_tensor.float()

        transformed_images = []
        for image in tensor:
            # Convert each image tensor (C, H, W) to PIL Image for transformation
            transformed_image = transform(image)  # Apply the transformations
            transformed_images.append(transformed_image)

        # 4. Stack the transformed images back into a batch
        preprocessed_tensor = torch.stack(transformed_images)  # Shape: (2500, 3, 224, 224)

        # 5. Check the shape of the preprocessed tensor
        print(preprocessed_tensor.shape)  

        for i in range(10) : 
            
            preprocessed_batch = preprocessed_tensor[i*250:(i+1)*250]
            preprocessed_batch = preprocessed_batch.to(device)

            # 4. Get the embeddings (feature maps)
            with torch.no_grad():  # Disable gradients for inference
                feature_maps = resnet(preprocessed_batch)  # Shape will be (batch_size, 512, 1, 1)

            # 5. Flatten the feature maps (optional)
            embeddings = feature_maps.view(feature_maps.size(0), -1)  # Flatten to shape (batch_size, embedding_size)

            embeds.append(embeddings)
        
        embeds = torch.vstack(embeds)
        domains[j]['features'] = embeds
        
        torch.save(embeds, f'stuff/train_embeds_{j+1}.pt')

/tmp/ipykernel_30289/100077929.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  domains[j]['features']  = torch.load(f'stuff/embeds_{j+1}.pt', map_location = device)


torch.Size([2500, 3, 224, 224])
torch.Size([2500, 3, 224, 224])
torch.Size([2500, 3, 224, 224])
torch.Size([2500, 3, 224, 224])
torch.Size([2500, 3, 224, 224])
torch.Size([2500, 3, 224, 224])


KeyboardInterrupt: 

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

class LWP:
    def __init__(self, n_classes, n_epochs=10):
        """
        Learning with Prototypes (LWP) with one prototype per class.
        
        Parameters:
        - n_classes: Number of classes, hence the number of prototypes
        - n_epochs: Number of epochs to train the model
        """
        self.n_classes = n_classes
        self.n_epochs = n_epochs
        self.prototypes = None  # Prototypes initialized later
    
    def fit(self, X, y):
        """
        Train the LWP model by iterating over the data and updating prototypes.
        
        Parameters:
        - X: Input dataset, shape (n_samples, n_features)
        - y: Labels, shape (n_samples,) corresponding to classes
        """
        # Initialize prototypes randomly from the data for each class
        self.prototypes = np.zeros((self.n_classes, X.shape[1]))
        
        # Initialize each prototype as the mean of the samples from that class
        for class_idx in range(self.n_classes):
            if isinstance(X[y == class_idx], torch.Tensor):
                class_samples = X[y == class_idx].to('cpu').numpy()
            else :
                class_samples = X[y == class_idx]
                
            if len(class_samples) > 0:
                self.prototypes[class_idx] = np.mean(class_samples, axis=0)
        
        # for epoch in range(self.n_epochs):
        #     print(f"Epoch {epoch + 1}/{self.n_epochs}")
        #     # Assign each sample to the nearest class prototype
        #     distances = euclidean_distances(X, self.prototypes)
        #     nearest_prototypes = np.argmin(distances, axis=1)
            
        #     # Update each prototype based on the samples assigned to it
        #     for class_idx in range(self.n_classes):
        #         assigned_samples = X[nearest_prototypes == class_idx]
                
        #         if len(assigned_samples) > 0:
        #             # Update the prototype as the mean of the assigned samples
        #             self.prototypes[class_idx] = np.mean(assigned_samples, axis=0)
    
    def predict(self, X):
        """
        Predict the closest prototype for each sample in X.
        
        Parameters:
        - X: Input dataset to predict, shape (n_samples, n_features)
        
        Returns:
        - predictions: Array of class labels (0, 1, ..., n_classes-1) for each sample
        """
        distances = euclidean_distances(X, self.prototypes)
        predictions = np.argmin(distances, axis=1)
        return predictions

In [ ]:
def sample_from_gmms(gmms, n_samples, sampling_probabilities, num_classes = 10):
    pseudo_features = []
    pseudo_labels = []
    
    for i in range(num_classes):
        # Determine the number of samples for this class based on its probability
        num_class_samples = int(n_samples * sampling_probabilities[i])
        
        # Sample from the ith GMM
        class_samples, _ = gmms[i].sample(num_class_samples)
        
        # Append the samples and corresponding class labels
        pseudo_features.append(class_samples)
        pseudo_labels.extend([i] * num_class_samples)
    
    # Concatenate the features and labels
    pseudo_features = np.concatenate(pseudo_features, axis=0)
    pseudo_labels = np.array(pseudo_labels)
    
    return pseudo_features, pseudo_labels

In [ ]:
from sklearn.mixture import GaussianMixture

num_classes = 10
buffer_size_per_class = 100

buffer_dataset = {'features': [], 'labels': []}
source_dataset = domains[0]
gmms = [None] * num_classes

model = LWP(n_classes=10, n_epochs=10)
model.fit(source_dataset['features'], source_dataset['labels'])

class_frequencies = [np.sum(source_dataset['labels'] == i) for i in range(num_classes)]
total_samples = np.sum(class_frequencies)
sampling_probabilities = np.array(class_frequencies) / total_samples

# Update GMM Models
for i in range(num_classes):
    gmms[i] = GaussianMixture(n_components=2, covariance_type='full')
    gmms[i].fit(source_dataset['features'][source_dataset['labels'] == i].to('cpu').numpy())
    
for i in range(num_classes):
    # Get all the samples of class 'i' from the current dataset
    class_samples = source_dataset['features'][source_dataset['labels'] == i].to('cpu').numpy()
    
    # Get the mean (centroid) of the class from the GMM
    class_mean = gmms[i].means_.mean(axis=0)  # Use the mean of the GMM components
    
    # Compute the distance of each sample to the class mean
    distances = np.linalg.norm(class_samples - class_mean, axis=1)
    
    # Select the 'buffer_size_per_class' least distant samples
    least_distant_indices = np.argsort(distances)[:buffer_size_per_class]
    
    # Add these least distant samples to the buffer
    buffer_dataset['features'].append(class_samples[least_distant_indices])
    buffer_dataset['labels'].append([i] * buffer_size_per_class)

# Convert buffer_dataset to numpy arrays
buffer_dataset['features'] = np.concatenate(buffer_dataset['features'], axis=0)
buffer_dataset['labels'] = np.concatenate(buffer_dataset['labels'], axis=0)

In [ ]:
pseudo_size = 1000
num_iters = 10

for i in range(1, 10) :
    curr_dataset = domains[i]['features'].to('cpu').numpy()
    curr_dataset_labels = model.predict(curr_dataset)
    
    pseudo_dataset = {'features': [], 'labels': []}
    
    pseudo_dataset['features'], pseudo_dataset['labels'] = sample_from_gmms(gmms, pseudo_size, sampling_probabilities, num_classes = 10)
    
    for j in range(num_iters):
        batch_size_pseudo = len(pseudo_dataset['features']) // num_iters
        batch_pseudo = pseudo_dataset['features'][j*batch_size_pseudo:(j+1)*batch_size_pseudo]
        batch_pseudo_labels = pseudo_dataset['labels'][j*batch_size_pseudo:(j+1)*batch_size_pseudo]
        
        batch_size_curr = len(curr_dataset) // num_iters
        batch_curr = curr_dataset[j*batch_size_curr:(j+1)*batch_size_curr]
        batch_curr_labels = curr_dataset_labels[j*batch_size_curr:(j+1)*batch_size_curr]
        
        # Combine the current dataset with the pseudo dataset
        batch = np.concatenate([batch_curr, batch_pseudo], axis=0)
        batch_labels = np.concatenate([batch_curr_labels, batch_pseudo_labels], axis=0)
        
        model.fit(batch, batch_labels)
        
    # Update GMM Models
    for i in range(num_classes):
        gmms[i] = GaussianMixture(n_components=2, covariance_type='full')
        gmms[i].fit(pseudo_dataset['features'][pseudo_dataset['labels'] == i])
    
    new_buffer = []
    new_buffer_labels = []
    
    for i in range(num_classes):
        # Get all the samples of class 'i' from the current dataset
        class_samples = pseudo_dataset['features'][pseudo_dataset['labels'] == i]
        
        # Get the mean (centroid) of the class from the GMM
        class_mean = gmms[i].means_.mean(axis=0)  # Use the mean of the GMM components
        
        # Compute the distance of each sample to the class mean
        distances = np.linalg.norm(class_samples - class_mean, axis=1)
        
        # Select the 'buffer_size_per_class' least distant samples
        least_distant_indices = np.argsort(distances)[:buffer_size_per_class]
        
        # Add these least distant samples to the buffer
        new_buffer.append(class_samples[least_distant_indices])
        new_buffer_labels.append([i] * buffer_size_per_class)

    new_buffer = np.concatenate(new_buffer, axis=0)
    new_buffer_labels = np.concatenate(new_buffer_labels, axis=0)

    # Convert buffer_dataset to numpy arrays
    buffer_dataset['features'] = np.concatenate([buffer_dataset['features'], new_buffer], axis=0)
    buffer_dataset['labels'] = np.concatenate([buffer_dataset['labels'], new_buffer_labels], axis=0)